<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/modelos_tradicionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/2010.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

# Agora, 'dados' contém os dados do arquivo XLSX


In [ ]:
# Exibe as primeiras linhas do DataFrame
print(dados.head())


   ID Código    Setor EconômicoB3     VM30Abr11   PLDez2010  LLDez2010  \
0   5  EALT4     Bens industriais  6.072750e+04     45809.0     5927.0   
1  21  ALPA4      Consumo cíclico  3.893923e+06   1310475.0   306341.0   
2  26  ABEV3  Consumo não cíclico  1.422082e+08  24361863.0  7561383.0   
3  28  AMER3      Consumo cíclico  2.382110e+06    225945.0    33587.0   
4  29  CBEE3    Utilidade pública  4.773374e+06   1583469.0   216092.0   

    ATDez2010   PTDez2010  VM30Abr11esc  PLDez2010esc  LLDez2010esc  \
0    237963.0    192154.0      0.424087      0.319904      0.041391   
1   2246603.0    936128.0      2.226423      0.749289      0.175156   
2  42678300.0  18316437.0      3.546250      0.607512      0.188558   
3   3212014.0   2986069.0      1.024863      0.097209      0.014450   
4   4313606.0   2730137.0      1.040166      0.345054      0.047089   

   ATDez2010esc  PTDez2010esc   ATDez2009  
0      1.661799      1.341895    143196.0  
1      1.284537      0.535249   1748959.

REGRESSÃO SEM ESCALA

tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [ ]:
from scipy.stats.mstats import winsorize

nivel_winsorizacao = 0.01  # 1% de winsorização

# Aplica a winsorização às variáveis
dados['PLDez2010_winsorizada'] = winsorize(dados['PLDez2010'], limits=[nivel_winsorizacao, nivel_winsorizacao])
dados['LLDez2010_winsorizada'] = winsorize(dados['LLDez2010'], limits=[nivel_winsorizacao, nivel_winsorizacao])
dados['VM30Abr11_winsorizada'] = winsorize(dados['VM30Abr11'], limits=[nivel_winsorizacao, nivel_winsorizacao])
# Variáveis winsorizadas agora contêm os valores winsorizados

In [ ]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PLDez2010_winsorizada', 'LLDez2010_winsorizada']]
y = dados['VM30Abr11_winsorizada']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Visualize os resultados do modelo
print(modelo.summary())

# Calcule o MedAE
y_pred = modelo.predict(X)
medae = np.median(np.abs(y - y_pred))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)


                              OLS Regression Results                             
Dep. Variable:     VM30Abr11_winsorizada   R-squared:                       0.948
Model:                               OLS   Adj. R-squared:                  0.947
Method:                    Least Squares   F-statistic:                     1674.
Date:                   Thu, 14 Sep 2023   Prob (F-statistic):          3.20e-119
Time:                           23:06:18   Log-Likelihood:                -3218.1
No. Observations:                    188   AIC:                             6442.
Df Residuals:                        185   BIC:                             6452.
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------


Em relação aos resultados apresentados no seu modelo de regressão:

O valor do teste de Durbin-Watson é 1.676, o que sugere uma leve autocorrelação positiva nos resíduos, mas não muito forte.
O valor-p associado ao teste de Jarque-Bera é praticamente zero (0.00), o que indica que os resíduos não seguem uma distribuição normal.

Interpretação DW:

Valor DW ≈ 2: Ausência de autocorrelação significativa nos resíduos (boa notícia).
Valor DW < 2: Autocorrelação positiva nos resíduos (erros adjacentes são correlacionados positivamente).
Valor DW > 2: Autocorrelação negativa nos resíduos (erros adjacentes são correlacionados negativamente).

Nesse caso específico, o valor de 17.74% indica que, em média, as previsões do modelo estão erradas em cerca de 17.74% do valor médio (ou preço) da variável de resposta. Isso ajuda a ter uma ideia do tamanho relativo dos erros do modelo em relação ao valor médio dos dados.

teste de heterocedasticidade de White. Se o valor-p for menor que um nível de significância escolhido (por exemplo, 0,05), você pode rejeitar a hipótese nula de homocedasticidade

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

# Calcule os resíduos do modelo
residuos = modelo.resid

# Realize o teste de White para heterocedasticidade
teste_white = het_white(residuos, X)

# Imprima os resultados do teste
print("Estatística do teste de White:", teste_white[0])
print("Valor-p do teste de White:", teste_white[1])

Estatística do teste de White: 113.73736510497207
Valor-p do teste de White: 6.642361768007011e-23


minimos quadrados ponderados (MQP), com pesos com base na variância dos erros

In [ ]:
import statsmodels.api as sm
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Calcule os resíduos (erros)
residuos = modelo.resid

# Calcule a variância dos resíduos
variancia_residuos = np.var(residuos)

# Calcule os pesos com base na variância dos resíduos (inverso da variância)
pesos = 1 / variancia_residuos

# Crie o modelo de regressão linear ponderada (WLS) com os pesos calculados
modelo_wls = sm.WLS(y, X, weights=pesos).fit()

# Visualize os resultados do modelo ponderado
print(modelo_wls.summary())

# Calcule os resíduos ponderados (erros ponderados)
y_pred_wls = modelo_wls.predict(X)
residuos_wls = y - y_pred_wls

# Calcule o MedAE ponderado
medae_wls = np.median(np.abs(residuos_wls))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled_wls = (medae_wls / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) ponderado:", medae_wls)
print("MedAE ponderado escalado pelo valor (ou preço) em percentagem:", medae_scaled_wls)


                            WLS Regression Results                            
Dep. Variable:              VM30Abr11   R-squared:                       0.963
Model:                            WLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     2411.
Date:                Thu, 14 Sep 2023   Prob (F-statistic):          3.13e-133
Time:                        23:10:36   Log-Likelihood:                -3216.0
No. Observations:                 188   AIC:                             6438.
Df Residuals:                     185   BIC:                             6448.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.752e+06   4.87e+05      3.595      0.0

REGRESSÃO QUANTÍLICA

In [12]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão quantílica
modelo = sm.QuantReg(y, X).fit(q=0.5)  # q=0.5 representa a mediana, ajuste conforme necessário

# Visualize os resultados do modelo
print(modelo.summary())
# Faça previsões com o modelo quantílico
y_pred_quantreg = modelo.predict(X)

# Calcule o MedAE para o modelo quantílico
medae_quantreg = np.median(np.abs(y - y_pred_quantreg))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled_quantreg = (medae_quantreg / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) quantílico:", medae_quantreg)
print("MedAE quantílico escalado pelo valor (ou preço) em percentagem:", medae_scaled_quantreg)


                         QuantReg Regression Results                          
Dep. Variable:              VM30Abr11   Pseudo R-squared:               0.7041
Model:                       QuantReg   Bandwidth:                   1.101e+06
Method:                 Least Squares   Sparsity:                    2.608e+06
Date:                Thu, 14 Sep 2023   No. Observations:                  188
Time:                        23:13:19   Df Residuals:                      185
                                        Df Model:                            2
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.734e-07    9.7e+04   2.82e-12      1.000   -1.91e+05    1.91e+05
PLDez2010      0.3167      0.010     32.294      0.000       0.297       0.336
LLDez2010      7.4244      0.070    106.250      0.000       7.287       7.562

The condition number is large, 2.54e+07. This might

REGRESSÃO RIDGE

In [25]:
import statsmodels.api as sm
import numpy as np
from scipy import stats

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão Ridge
alpha = 1.0  # O valor de alpha controla a força da regularização (ajuste conforme necessário)
modelo = sm.OLS(y, X).fit_regularized(alpha=alpha, L1_wt=0.0)  # L1_wt=0.0 indica Ridge

# Visualize os coeficientes do modelo Ridge
coeficientes = modelo.params
print("Coeficientes do modelo Ridge:")
print(coeficientes)

# Calcule os resíduos
residuos = y - modelo.predict(X)

# Calcule os graus de liberdade
n = len(y)
p = len(X.columns) - 1  # Número de variáveis independentes

# Calcule a matriz de covariância dos coeficientes
cov_matrix = np.linalg.inv(np.dot(X.T, X))
std_err = np.sqrt(np.diagonal(cov_matrix))

# Obtenha as estatísticas t
estatisticas_t = coeficientes / (std_err / np.sqrt(n - p - 1))

# Calcule os valores de p-valor
p_valores = 2 * (1 - stats.t.cdf(np.abs(estatisticas_t), df=n - p - 1))

# Crie um DataFrame para visualizar os coeficientes e os valores de p-valor
import pandas as pd
coeficientes_pvalores = pd.DataFrame({'Coeficientes': coeficientes[1:], 'p-valor': p_valores[1:]}, index=X.columns[1:])

print(coeficientes_pvalores)

# Calcule o R²
r_squared = modelo.rsquared
print("R²:", r_squared)

# Calcule o MedAE
medae = np.median(np.abs(residuos))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)


Coeficientes do modelo Ridge:
[8.59259706e+05 2.02316730e-01 8.45166655e+00]
           Coeficientes  p-valor
PLDez2010      0.202317      0.0
LLDez2010      8.451667      0.0


AttributeError: ignored